In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from skimage.metrics import structural_similarity as ssim

In [2]:
!cd /content/
!unzip /content/drive/MyDrive/CV_proj/train.zip -d .

Archive:  /content/drive/MyDrive/CV_proj/train.zip
   creating: ./train/
  inflating: ./train/keshavarzi_100.txt  
  inflating: ./train/keshavarzi_101.jpg  
  inflating: ./train/keshavarzi_101.txt  
  inflating: ./train/keshavarzi_102.jpg  
  inflating: ./train/keshavarzi_102.txt  
  inflating: ./train/keshavarzi_103.jpg  
  inflating: ./train/keshavarzi_103.txt  
  inflating: ./train/keshavarzi_104.jpg  
  inflating: ./train/keshavarzi_104.txt  
  inflating: ./train/keshavarzi_105.jpg  
  inflating: ./train/keshavarzi_105.txt  
  inflating: ./train/keshavarzi_106.jpg  
  inflating: ./train/keshavarzi_106.txt  
  inflating: ./train/keshavarzi_180.jpg  
  inflating: ./train/keshavarzi_180.txt  
  inflating: ./train/keshavarzi_181.jpg  
  inflating: ./train/keshavarzi_181.txt  
  inflating: ./train/keshavarzi_182.jpg  
  inflating: ./train/keshavarzi_182.txt  
  inflating: ./train/keshavarzi_183.jpg  
  inflating: ./train/keshavarzi_183.txt  
  inflating: ./train/keshavarzi_184.jpg  
  i

In [ ]:
from keras.layers.serialization import activation
# input_layer = layers.Input(shape=(None, None, 3))
# resize_layer = layers.Resizing(300, 300)(input_layer)
vgg = tf.keras.applications.VGG16(classes=3, include_top=False, input_shape=(300, 300, 3))
# for l in vgg.layers:
#   if l.name == 'block4_conv3':
#     x = l
#     break

classifier_1 = layers.Conv2D(28, (3, 3), name='classifier_1', padding='same')(vgg.output)
# classifier_1 = layers.MaxPooling2D((37, 37))(classifier_1)
r1 = layers.Reshape((-1, 28))(classifier_1)

vgg_pool = layers.MaxPooling2D()(vgg.output)

conv_layer1 = layers.Conv2D(1024, (3, 3), padding='same')(vgg_pool)
conv_layer2 = layers.Conv2D(1024, (1, 1), padding='same')(conv_layer1)
classifier_2 = layers.Conv2D(42, (3, 3), name='classifier_2', padding='same')(conv_layer2)
classifier_2 = layers.MaxPooling2D((18, 18))(classifier_2)
r2 = layers.Reshape((-1, 42))(classifier_2)

pool_layer1 = layers.MaxPooling2D()(conv_layer2)
conv_layer3 = layers.Conv2D(256, (1, 1), padding='same')(pool_layer1)
conv_layer4 = layers.Conv2D(512, (3, 3), padding='same')(conv_layer3)
classifier_3 = layers.Conv2D(42, (3, 3), name='classifier_3', padding='same')(conv_layer4)
classifier_3 = layers.MaxPooling2D((9, 9))(classifier_3)
r3 = layers.Reshape((-1, 42))(classifier_3)

pool_layer2 = layers.MaxPooling2D()(conv_layer4)
conv_layer5 = layers.Conv2D(128, (1, 1), padding='same')(pool_layer2)
conv_layer6 = layers.Conv2D(256, (3, 3), padding='same')(conv_layer5)
classifier_4 = layers.Conv2D(42, (3, 3), padding='same', name='classifier_4')(conv_layer6)
classifier_4 = layers.MaxPooling2D((4, 4))(classifier_4)
r4 = layers.Reshape((-1, 42))(classifier_4)

pool_layer3 = layers.MaxPooling2D()(conv_layer6)
conv_layer7 = layers.Conv2D(128, (1, 1), padding='same')(pool_layer3)
conv_layer8 = layers.Conv2D(256, (3, 3), padding='same')(conv_layer7)
classifier_5 = layers.Conv2D(28, (3, 3), padding='same', name='classifier_5')(conv_layer8)
classifier_5 = layers.MaxPooling2D()(classifier_5)
r5 = layers.Reshape((-1, 28))(classifier_5)

conv_layer9 = layers.Conv2D(128, (1, 1), padding='same')(conv_layer8)
conv_layer10 = layers.Conv2D(256, (3, 3), padding='same')(conv_layer9)
classifier_6 = layers.Conv2D(28, (3, 3), padding='same', name='classifier_6')(conv_layer10)
classifier_6 = layers.MaxPooling2D()(classifier_6)
r6 = layers.Reshape((-1, 28))(classifier_6)

conct_layer = layers.Concatenate()([r1, r2, r3, r4, r5, r6])
# rl = conct_layer
# class_prediction = layers.Conv2D(3, (3, 3), padding='same', activation='softmax')(rl)
# box_prediction = layers.Conv2D(8, (3, 3), padding='same', activation='linear')(rl)

# output_layer = layers.Concatenate()([class_prediction, box_prediction])
# output_layer = layers.Conv2D(12, (37,37))(conctl)
# output_layer = layers.Reshape((-1, 12))(output_layer)

model = Model(inputs=vgg.input, outputs=[conct_layer])
model.summary()

In [160]:
def iou_loss(y_true, y_pred):
    # Extract the ground truth and predicted bounding boxes
    true_box = y_true[..., :4]
    pred_box = y_pred[..., :4]

    # Calculate the intersection and union areas
    intersection = tf.reduce_sum(tf.minimum(true_box, pred_box), axis=-1)
    union = tf.reduce_sum(tf.maximum(true_box, pred_box), axis=-1)

    # Calculate the IOU
    iou = intersection / (union + tf.keras.backend.epsilon())

    # Calculate the IOU loss
    iou_loss = 1.0 - iou

    return iou_loss
# def custom_loss(y_pred, y_gt, smooth=1):
#   return

In [266]:
base_dir = '/content/'
train_dir = base_dir + 'train/'

images, labels = [], []
for (dirpath, dirname, filenames) in os.walk(train_dir):
  for filename in filenames:
    if filename.split('.')[1] != 'txt':
      path = dirpath + filename
      img = cv2.imread(path)
      img = cv2.resize(img, (300, 300))
      images.append(img)
      txt_path = dirpath + filename.split('.')[0] + '.txt'
      with open(txt_path, 'r') as f:
        lines = f.readlines()
        txt = []
        # for l in lines:
        #   txt.append(list(map(float,l.split())))
        a = list(map(float,lines[0].split()))[1::]
        b = list(map(float,lines[1].split()))[1::]
        txt.append([1., 0.] + [float(it) for it in a])
        txt.append([0., 1.] + [float(it) for it in b])
      txt = np.array(txt)
      # txt = txt.reshape((-1, 12))
      labels.append(txt)

images = np.array(images)
labels = np.array(labels)

In [267]:
print(images.shape)
print(labels.shape)
print(labels[0])

(1300, 300, 300, 3)
(1300, 2, 6)
[[1.         0.         0.4990099  0.71473354 0.8019802  0.09404389]
 [0.         1.         0.22178218 0.89341693 0.16831683 0.04702194]]


In [268]:
images[0: 600] = images[0:600]/255.

In [269]:
images[600::] = images[600::]/255.

In [249]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, fill_mode='nearest', validation_split=0.2)

datagen.fit(images)

In [ ]:
model.compile(optimizer='adam', loss=iou_loss, metrics=['accuracy'])
model.fit(images, labels, epochs=20, validation_split=0.2, batch_size=16, shuffle=True)

In [ ]:
test_images = [cv2.imread(base_dir+'test/2.jpg'), cv2.imread(base_dir+'test/8.jpg')]
for i in range(len(test_images)):
  test_images[i] = cv2.resize(test_images[i], (300, 300))
test_images = np.array(test_images)
print(test_images.shape)
print(test_images[0].shape)
prediction = model.predict(test_images)

In [ ]:
prediction[1]

In [200]:
class NMSLayer(layers.Layer):
  def __init__(self, iou_threshold=0.5):
        super(NMSLayer, self).__init__()
        self.iou_threshold = iou_threshold

  def call(self, inputs):
      # Extract the bounding box predictions and scores from the inputs
      boxes, scores = inputs[0], inputs[1]

      # Apply non-maximum suppression
      selected_indices = tf.image.non_max_suppression(
          boxes, scores, max_output_size=100, iou_threshold=self.iou_threshold
      )

      # Gather the selected bounding boxes and scores
      selected_boxes = tf.gather(boxes, selected_indices)
      selected_scores = tf.gather(scores, selected_indices)

      # Return the selected bounding boxes and scores
      return [selected_boxes, selected_scores]

In [285]:
input_layer = layers.Input(shape=(300, 300, 3))
resnet = tf.keras.applications.ResNet50(classes=3, include_top=False, input_shape=(300, 300, 3))(input_layer)
conv1 = layers.Conv2D(1024, (3, 3))(resnet)
conv2 = layers.Conv2D(512, (3, 3))(conv1)
conv3 = layers.Conv2D(256, (3, 3))(conv2)
flat = layers.Flatten()(conv3)
dense = layers.Dense(4, activation='softmax')(flat)
classifier_model = Model(inputs=input_layer, outputs=dense)
# classifier_model.summary()
#############################
vgg = tf.keras.applications.VGG16(classes=3, include_top=False, input_shape=(300, 300, 3))(input_layer)
rconv1 = layers.Conv2D(1024, (3, 3), padding='same')(vgg)
rconv2 = layers.Conv2D(256, (3, 3), padding='same')(rconv1)
f1 = layers.Flatten()(rconv2)
d1 = layers.Dense(4, activation='linear')(f1)
rconv3 = layers.Conv2D(1024, (3, 3), padding='same')(rconv2)
rconv4 = layers.Conv2D(512, (3, 3), padding='same')(rconv3)
f2 = layers.Flatten()(rconv4)
d2 = layers.Dense(4, activation='linear')(f2)
conct = layers.Concatenate()([d1, d2])
resh = layers.Reshape((-1, 4))(conct)

regression_model = Model(inputs=input_layer, outputs=resh)
# regression_model.summary()
###############################
output = layers.Concatenate()([layers.Reshape((2,2))(classifier_model.output), regression_model.output])
# output = layers.Reshape((2,1))(output)
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_145"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_242 (InputLayer)         [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, 9, 9, 512)    14714688    ['input_242[0][0]']              
                                                                                                  
 conv2d_1276 (Conv2D)           (None, 9, 9, 1024)   4719616     ['vgg16[0][0]']                  
                                                                                                  
 resnet50 (Functional)          (None, 10, 10, 2048  23587712    ['input_242[0][0]']      

In [ ]:
model.fit(images, labels, epochs=20, validation_split=0.2, batch_size=16, shuffle=True)

Epoch 1/20
43/65 [==================>...........] - ETA: 14s - loss: 27.4378 - accuracy: 0.0400